# IAPI Controller Demo

This notebook to demonstrate how we can use pythonnet (http://pythonnet.github.io/) to call Thermo's Instrument Application Programming Interface
 (https://github.com/thermofisherlsms/iapi) libraries from Python and uses a controller to handle the scans.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
import glob
import datetime
import time
from pathlib import Path

In [ ]:
base_folder = 'C:\\Users\\joewa\\University of Glasgow\\Vinny Davies - CLDS Metabolomics Project\\'
vimms_folder = 'C:\\Users\\joewa\\Work\\git\\vimms'
base_folder = 'C:\\Users\\Vinny\\OneDrive - University of Glasgow\\CLDS Metabolomics Project'
vimms_folder = 'C:\\Users\\Vinny\\work\\vimms'

In [ ]:
import sys
sys.path.append('C:\\Users\\joewa\\Work\\git\\vimms')
sys.path.append('C:\\Users\\Vinny\\work\\vimms')

from vimms.MassSpec import IndependentMassSpectrometer, IAPIMassSpectrometer
from vimms.Controller import IdleController, SimpleMs1Controller, TopNController, RoiController, HybridController
from vimms.Environment import Environment, IAPIEnvironment
from vimms.Common import *

In [ ]:
ref_dir = vimms_folder + '\\Interface\\FusionLibrary\\bin\\Debug\\netstandard2.0'
if ref_dir not in sys.path:
    sys.path.append(ref_dir)

In [ ]:
# filename = base_folder + '\\Data\\multibeers_urine_data\\beers\\fragmentation\\mzML\\Beer_multibeers_1_T10_POS.mzML'
# show_console_logs = True

In [ ]:
filename = None
show_console_logs = False

## Test with Top-N Controller

### Verbose mode

In [ ]:
# ionisation_mode = POSITIVE
# N = 10
# isolation_width = 1
# mz_tol = 10
# rt_tol = 15
# min_ms1_intensity = 1.75E5
# controller = TopNController(ionisation_mode, N, isolation_width, mz_tol, rt_tol, min_ms1_intensity)

In [ ]:
# mass_spec = IAPIMassSpectrometer(ionisation_mode, ref_dir, filename=filename, show_console_logs=show_console_logs)

In [ ]:
# pbar = False
# set_log_level_debug()
# max_time = 5

In [ ]:
# env = IAPIEnvironment(mass_spec, controller, max_time, progress_bar=pbar, out_dir='results', out_file='iapi_test.mzML')
# env.run()

### Normal mode

In [ ]:
ionisation_mode = POSITIVE
N = 10
isolation_width = 1
mz_tol = 10
rt_tol = 15
min_ms1_intensity = 1.75E5
controller = TopNController(ionisation_mode, N, isolation_width, mz_tol, rt_tol, min_ms1_intensity)

In [ ]:
mass_spec = IAPIMassSpectrometer(ionisation_mode, ref_dir, filename=filename, show_console_logs=show_console_logs)

In [ ]:
pbar = True
set_log_level_warning()
max_time = 300

In [ ]:
env = IAPIEnvironment(mass_spec, controller, max_time, progress_bar=pbar, out_dir='results', out_file='iapi_test_TopN.mzML')
env.run()

## Test with ROI Controller

In [ ]:
ionisation_mode = POSITIVE
N = 10
isolation_width = 1
mz_tol = 10
rt_tol = 15
min_ms1_intensity = 1.75E5
min_roi_intensity = 5000
min_roi_length = 10
score_method = 'Top N'

controller = RoiController(ionisation_mode, isolation_width, mz_tol, min_ms1_intensity,
                           min_roi_intensity, min_roi_length, score_method, N, rt_tol)

In [ ]:
mass_spec = IAPIMassSpectrometer(ionisation_mode, ref_dir, filename=filename, show_console_logs=show_console_logs)

In [ ]:
pbar = True
set_log_level_warning()
max_time = 300

In [ ]:
env = IAPIEnvironment(mass_spec, controller, max_time, progress_bar=pbar, out_dir='results', out_file='iapi_test_ROI.mzML')
env.run()

## Test with Full Purity Controller

In [ ]:
ionisation_mode = POSITIVE
isolation_window = [1]   # the isolation window in Dalton around a selected precursor ion
N = [16]
rt_tol = [10]
mz_tol = [10]
min_ms1_intensity = 1.75E5
scan_param_changepoints = None
rt_range = [(0, 300)]

In [ ]:
n_purity_scans = N[0]
purity_shift = 0.1
purity_threshold=1

In [ ]:
# these settings change the Mass Spec type. They arent necessary to run the Top-N ROI Controller
isolation_transition_window = 'gaussian'
isolation_transition_window_params = [0.5]

In [ ]:
purity_add_ms1 = True # this seems to be the broken bit
purity_randomise = True

In [ ]:
mass_spec = IAPIMassSpectrometer(ionisation_mode, ref_dir, filename=filename, show_console_logs=show_console_logs)

In [ ]:
controller = HybridController(mass_spec, N, scan_param_changepoints, isolation_window, mz_tol, rt_tol, min_ms1_intensity, n_purity_scans, purity_shift, purity_threshold)

In [ ]:
pbar = True
set_log_level_warning()
max_time = rt_range[0][1]

In [ ]:
env = IAPIEnvironment(mass_spec, controller, max_time, progress_bar=pbar, out_dir='results', out_file='iapi_test_purity.mzML')
env.run()